In [7]:
# 측정하는 cell
%reset
%cd C:\Users\ChungkiLee\Documents\GitHub\webcam

import cv2
import numpy as np
import datetime
import sys
import time
import pylab
import os
from matplotlib import pyplot as plt
import shelve
# from imutils import face_utils
# import imutils
# import dlib

def convertToRGB(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def valueToExtract(img):
    
    b, g, r = cv2.split(img)
    tmp2 = np.mean(r)
    tmp3 = np.mean(g)
    tmp4 = np.mean(b)
    return tmp2, tmp3, tmp4

def column(matrix, i):
    return [row[i] for row in matrix]

def resource_path(relative_path):
    """ Get absolute path to resource, works for dev and for PyInstaller """
    try:
        # PyInstaller creates a temp folder and stores path in _MEIPASS
        base_path = sys._MEIPASS
    except Exception:
        base_path = os.path.abspath(".")

    return os.path.join(base_path, relative_path)

def get_subface_coord(face_rect, fh_x, fh_y, fh_w, fh_h):
    x, y, w, h = face_rect
    return [int(x + w * fh_x - (w * fh_w / 2.0)),
            int(y + h * fh_y - (h * fh_h / 2.0)),
            int(w * fh_w),
            int(h * fh_h)]

def preset_img_size(face_rect, wn):
    x, y, w, h = face_rect[0]
    w = wn
    
    h = int(w*1.1);
    faces_tmp = [[x, y, w, h]]
    return faces_tmp


# Create a VideoCapture object
cap = cv2.VideoCapture(0)
frameRate = 30.0
cap.set(cv2.CAP_PROP_FPS, frameRate)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720) # 720, 1080
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280) # 1280, 1980
fps = int(cap.get(cv2.CAP_PROP_FPS))
print("The frame rate is: " + str(fps))

dpath = resource_path('haarcascade_frontalface_alt.xml') #'haarcascade_frontalface_default.xml', 'haarcascade_frontalface_alt.xml'
face_cascade = cv2.CascadeClassifier(dpath)

# Check if camera opened successfully
if (cap.isOpened() == False): 
    print("Unable to read camera feed")

# Default resolutions of the frame are obtained.The default resolutions are system dependent.
# We convert the resolutions from float to integer.
frame_width  = int(cap.get(3))
frame_height = int(cap.get(4))

# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
fourcc = cv2.VideoWriter_fourcc(*'XVID') # XVID
tmp1 = str(datetime.datetime.now())
tmp1 = tmp1.replace(":", "_").replace(".", "_").replace(" ", "_").replace("-", "_")
tmp2 = 'C:/Users/ChungkiLee/Documents/GitHub/webcam/previewimgs/outpy'
tmp3 = tmp2 + tmp1[:19] +'.mp4'
tmp4 = tmp2 + tmp1[:19] +'.out'
out = cv2.VideoWriter(tmp3, fourcc, frameRate, (frame_width,frame_height))
filename = tmp4
del(tmp1, tmp2, tmp3, tmp4)

imageEndingSeconds = 10;
mg = 10; sig_y = []; sig_x =[];
shooting = 'N'
cnt = 0
a= 0.0; b=0.0; c= 0.0;
sig_x = [] ; sig_y = [];
faces = [];
faces_tmp = [[0, 0, 0, 0]];
imgs = [];
a = datetime.datetime.now()

while(True):
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    if ret == True: 
        b = datetime.datetime.now()
        c = b - a           
        if cnt > 90:
            
            # 얼굴영역 처리하는 곳
            for (x,y,w,h) in faces:
                cv2.rectangle(frame,(x-mg-2,y-mg-2),(x+w+mg+1,y+h+mg+1),(0,255,0),2)
                roi_gray = gray[y:y+h, x:x+w]
                roi_color = frame[y-mg:y+h+mg, x-mg:x+w+mg]
                common_color = frame
                red,green,blue = valueToExtract(roi_color)
                redC,greenC,blueC = valueToExtract(common_color)
                imgs.append(roi_color)
                sig_y.append([red,green,blue,redC,greenC,blueC])
                sig_x.append((c.seconds-imagePreparSeconds) + c.microseconds/1000000)

            # 신호처리하는 곳
            if len(sig_x) > 3:
                x = np.array(sig_x)
                y = -(np.array(sig_y)[:, 1])
#                 y = -(np.array(sig_y)[:, 1]/(np.array(sig_y)[:, 4] - np.array(sig_y)[:, 1]))
                xx = (frame_width)*(x - x.min()) / (imageEndingSeconds - x.min())
                yy = (frame_height/7)*(y - y.min()) / (y.max() - y.min())+frame_height*1/6
                pts = np.array([[x_, y_] for x_, y_ in zip(xx,yy)], np.int32)
                frame = cv2.polylines(frame,[pts[:cnt-90]],False,(0,255,255), 2)
                
            cv2.putText(frame, "image number: %s" % str(cnt-90), (10, 25), cv2.FONT_HERSHEY_PLAIN, 2, (100, 255, 100))
            cv2.putText(frame, "recording time(sec): %s" % str(int(c.seconds-imagePreparSeconds)), (10, 50), cv2.FONT_HERSHEY_PLAIN, 2, (100, 255, 100))
            
            # Write the frame into the file 'output.avi'
            out.write(frame) 
            
            if ((c.seconds-imagePreparSeconds) + c.microseconds/1000000) > imageEndingSeconds:
                break
        else:
            # 얼굴 인식 하는곳
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            k = 31
            gray = cv2.GaussianBlur(gray, (k, k), 0)
#             gray = cv2.equalizeHist(gray)   
            gray = cv2.pyrDown( gray )
            gray = cv2.pyrDown( gray )
            faces = face_cascade.detectMultiScale(gray, 1.3, 3, flags = cv2.CASCADE_SCALE_IMAGE)
            faces = faces * 4
            if len(faces) == 1:
                faces_tmp = (faces_tmp + faces)//2
#                 print(faces_tmp[0][2])
                faces2 = preset_img_size(faces_tmp, int(faces_tmp[0][2]*1.1))
                faces = np.array(faces2)
            
            for (x,y,w,h) in faces:
                cv2.rectangle(frame,(x-mg-2,y-mg-2),(x+w+mg+1,y+h+mg+1),(0,255,0),2)
            
            cv2.putText(frame, "Ready to the 90th frame: %s" % str(cnt), (10, 25), cv2.FONT_HERSHEY_PLAIN, 2, (100, 255, 100))
            cv2.putText(frame, "recording time(sec): %s" % str(c.seconds), (10, 50), cv2.FONT_HERSHEY_PLAIN, 2, (100, 255, 100))
            imagePreparSeconds = ((c.seconds) + c.microseconds/1000000)
            
        cnt += 1
            
        # Display the resulting frame    
        cv2.imshow('frame',frame)
        
        # Press Q on keyboard to stop recording
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    # Break the loop
    else:
        break

# When everything done, release the video capture and video write objects
cap.release()
out.release()
 
# Closes all the frames
cv2.destroyAllWindows()

my_shelf = shelve.open(filename, 'n')
saving_files = ['imgs', 'sig_x', 'sig_y']
for key in saving_files:
    try:
        my_shelf[key] = globals()[key]
    except TypeError:
        print('error shelvinf : {0}'.format(key))
my_shelf.close()

C:\Users\ChungkiLee\Documents\GitHub\webcam
The frame rate is: 30


In [6]:
len(sig_x), sig_x

(261,
 [0.05563199999999957,
  0.10345999999999955,
  0.1463989999999995,
  0.17957999999999952,
  0.20885999999999955,
  0.23618799999999956,
  0.26546799999999954,
  0.29572499999999957,
  0.3279329999999996,
  0.36111599999999955,
  0.39430099999999957,
  0.4274859999999996,
  0.4606679999999995,
  0.49385199999999957,
  0.5280129999999996,
  0.5628579999999995,
  0.5953219999999996,
  0.6343919999999996,
  0.6655959999999995,
  0.6997529999999995,
  0.7329359999999996,
  0.7641889999999996,
  0.7963789999999995,
  0.8295869999999995,
  0.8627459999999996,
  0.8959549999999995,
  0.9291539999999996,
  0.9662009999999995,
  0.9955119999999995,
  1.0296419999999995,
  1.0716379999999996,
  1.0989629999999995,
  1.1301679999999994,
  1.1633799999999996,
  1.1965359999999996,
  1.2297469999999995,
  1.2639059999999995,
  1.2960879999999995,
  1.3341509999999996,
  1.3644339999999995,
  1.4034469999999994,
  1.4337039999999996,
  1.4639599999999995,
  1.5026169999999994,
  1.531895999999